In [1]:
# PSU GEOG 489 Final Project Assignment
# Ryan Prasad
# rhprasad@outlook.com

# The task for this introductory assignment is to create a cloud free image composite with the geemap library.
# AOI is of Ocala National Forest, Tampa. Fantastic hiking + camping here during winter season.
# Pro tip: bring your bear bag to hang from a tree or bear-proof canister! 

import ee
import geemap

Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

<IPython.core.display.Javascript object>

In [12]:
# Upload your shapefile to your Earth Engine Assets as a table, then call it in the line below
ocalanf_boundary = ee.FeatureCollection("users/ryanprasad/Ocala_National_Forest")

landsat = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') # Grab Landsat 8 Collection 1 Tier 1 Surface Reflectance Imagery
    .filter(ee.Filter.eq('WRS_PATH', 16)) # these specify the location of the image collection we want
    .filter(ee.Filter.eq('WRS_ROW', 40))
    .filterDate('2020-05-01', '2020-07-30') # these specify the data range of the image collection, in this case, Summer '20
    .mean() # create a mean composite image from the image collection
    .clip(ocalanf_boundary) # CLip the image to the polygon
)

# CIR and RGB visualization parameters
landsat_vis_cir = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

landsat_vis_rgb = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}

# Add CIR, RGB, and polygon layers to map
Map.addLayer(landsat, landsat_vis_cir, 'Landsat 8 CIR')
Map.addLayer(landsat, landsat_vis_rgb, 'Landsat 8 RGB')
Map.addLayer(ocalanf_boundary, {}, "Ocala National Forest Boundary")

# Center and scale the map
Map.setCenter(-81.68067, 29.24847, 10)

In [5]:
# Define a function that masks out the clouds and cloud shadows using the landsat 8 QA band

def mask_clouds_landsat8(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = (1 << 3) # 1000 in base 2
  cloudsBitMask = (1 << 5) # 100000 in base 2

  # Get the pixel QA band.
  qa = image.select('pixel_qa')

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  # Mask image with clouds and shadows.
  return image.updateMask(mask)


In [6]:
# Make a second mean image composite with the clouds and shadows masked out using the function defined above

landsat_cloud_free = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') # Grab Landsat 8 Collection 1 Tier 1 Surface Reflectance Imagery
    .filter(ee.Filter.eq('WRS_PATH', 16))
    .filter(ee.Filter.eq('WRS_ROW', 40))
    .filterDate('2020-05-01', '2020-07-30')
    .map(mask_clouds_landsat8)
    .mean() # create a mean composite image from the image collection
    .clip(ocalanf_boundary) # CLip the image to the polygon
)

In [8]:
# Make a second plot for the cloud-free image composite

Map2 = geemap.Map()
Map2

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [9]:
# Add CIR, RGB, and polygon layers to map
Map2.addLayer(landsat_cloud_free, landsat_vis_cir, 'Landsat 8 CIR')
Map2.addLayer(landsat_cloud_free, landsat_vis_rgb, 'Landsat 8 RGB')
Map2.addLayer(ocalanf_boundary, {}, "Ocala National Forest Boundary")

# Center and scale the map
Map2.setCenter(-81.68067, 29.24847, 10)

In [15]:
# Create a split pane map to compare the cloudy and cloud-free image composites
Map3 = geemap.Map()

left_layer = geemap.ee_tile_layer(landsat, landsat_vis_rgb, 'Clouded')
right_layer = geemap.ee_tile_layer(landsat_cloud_free, landsat_vis_rgb, 'Cloud Free')

Map3.split_map(left_layer, right_layer)
Map3.setCenter(-81.68067, 29.24847, 10)
Map3

Map(center=[29.24847, -81.68067], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …